# INFO6105 Data Science Engineering Methods & Tools

# Gradient Boosting

Navaneeta Naik

NUID: 001027107

# Abstract

To briefly describe how gradient boosting differs from bagging and Implement gradient boosting as invoked in scikit-learn, and evaluate classifier accuracy for the Titanic dataset

Boosting & Bagging

1.  Gradient boosting is a stage-wise additive model that generates learners during the learning process. It builds the model in a stage-wise fashion where as Bagging is the aggregation of machine learning models trained on bootstrap samples


2.  Bagging is a train and keep technique where as Boosting is a train and evaluate technique


3.  Bagging technique is repeated to create many models and every model is trained in parallel as compared to boosting  techniques where the model is trained sequentially from the previous base models, so each time it corrects the mistakes of the previous ones.


4.  Bagging solves the difficulty of the single model is over-fitting. Boosting doesn’t help to avoid over-fitting



# Import all necessary packages

In [90]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from pandas.api.types import is_string_dtype,is_numeric_dtype
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
%matplotlib inline

# Reading in data

In [91]:
path = 'C:\\Users\\navan\\Documents\\DS\\Assignment2'

In [92]:
'''Read in training and test datasetes. Here, for simplicity, we use
train and test instead of train and valid and test'''

trn_df = pd.read_csv(f'{path}/titanic_trn_4_ml.csv')
test_df = pd.read_csv(f'{path}/titanic_test_4_ml.csv')

print(trn_df.shape)
print(test_df.shape)

(1100, 13)
(209, 13)


In [93]:
trn_df.head(2)

,PassengerId,Pclass,Name,Sex,SibSp,Parch,Ticket,Embarked,Age_na,Fare_na,Age,Fare,Survived
0,1,3,135,2,1,0,629,3,False,False,-0.574896,-0.502155,0
1,2,1,245,1,1,0,713,1,False,False,0.659705,0.747957,1


In [94]:
test_df.head(2)

,PassengerId,Pclass,Name,Sex,SibSp,Parch,Ticket,Embarked,Age_na,Fare_na,Age,Fare,Survived
0,1101,3,47,2,0,0,106,3,False,False,-0.343408,-0.489547,1
1,1102,3,6,2,0,0,146,3,False,False,0.196729,-0.203944,0


# Exploring over and under fit models with KNN

In [95]:
#Create X and Y for train
x_train = trn_df.drop('Survived',axis = 1)
y_train = trn_df['Survived']
print(x_train.shape,y_train.shape)

(1100, 12) (1100,)


In [96]:
#Create X and Y for test set

x_test = test_df.drop('Survived',axis = 1)
y_test = test_df['Survived']
print(x_test.shape,y_test.shape)

(209, 12) (209,)


In [97]:
# Train AND predict on the TRAINING set with k = 1

knn_model = KNeighborsClassifier(n_neighbors = 1)
knn_model.fit(x_train, y_train)
y_pred = knn_model.predict(x_train)

In [98]:
#Evaluate training set accuracy
accuracy_score(y_train,y_pred)

1.0

In [99]:
#Evaluate test set accuracy
y_pred_test = knn_model.predict(x_test)

In [100]:
#Evaluate test set accuracy
accuracy_score(y_test,y_pred_test)

0.5406698564593302

Training set accuracy (=1.0) is much greater than test set 
accuracy (0.54). This means that we have an overfit model!
Let's try reducing the value of K to see if we get better 
test set accuracy

In [101]:
# Train AND predict on the TRAINING set with k = 5
knn_model_I = KNeighborsClassifier(n_neighbors = 5)
knn_model_I.fit(x_train, y_train)
y_pred = knn_model_I.predict(x_train)
accuracy_score(y_train,y_pred)

0.7381818181818182

In [102]:
#Evaluate test set accuracy with the same model,i .e. K = 5 model
y_pred_test = knn_model_I.predict(x_test)
accuracy_score(y_test,y_pred_test)

0.5454545454545454

We see an incremental increase in accuracy for test set!However, the
training set accuracy has also fallen. This means, our model is
underfit ! Let's try an intermediate value of K next.

In [103]:
# Train AND predict on the TRAINING set with k = 3
knn_model_III = KNeighborsClassifier(n_neighbors = 3)
knn_model_III.fit(x_train, y_train)
y_pred = knn_model_III.predict(x_train)
accuracy_score(y_train,y_pred)

0.8072727272727273

In [104]:
#Evaluate test set accuracy with the same model, i.e, k = 3 model
y_pred_test = knn_model_III.predict(x_test)
accuracy_score(y_test,y_pred_test)

0.5550239234449761

'''Thus, K =3 seems to be right model here. Try bigger values of K. What do you find?One reason why our model may behave differently from what you would expect is because of the small dataset !!!'''

In [105]:
scaler = MinMaxScaler()
x_train_scale = scaler.fit_transform(x_train)
x_test_scale = scaler.transform(x_test)
x_train_1, x_validation_1, y_train_1, y_validation_1 = train_test_split(x_train_scale, y_train, random_state=0)

# Hyperparameter Tuning

In [106]:
learning_rates = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]

for learning_rate in learning_rates:
    gb = GradientBoostingClassifier(n_estimators=50, learning_rate=learning_rate, max_features=2, max_depth=2, 
                                        random_state=0)
    gb.fit(x_train_1, y_train_1)
    
    print("Learning rate: ", learning_rate)
    print("Training Accuracy Score: {:}".format(gb.score(x_train_1, y_train_1)))
    print("Validation Accuracy Score: {:}".format(gb.score(x_validation_1, y_validation_1)))

Learning rate:  0.05
Training Accuracy Score: 0.7878787878787878
Validation Accuracy Score: 0.730909090909091
Learning rate:  0.075
Training Accuracy Score: 0.8109090909090909
Validation Accuracy Score: 0.7163636363636363
Learning rate:  0.1
Training Accuracy Score: 0.8193939393939393
Validation Accuracy Score: 0.7090909090909091
Learning rate:  0.25
Training Accuracy Score: 0.833939393939394
Validation Accuracy Score: 0.6981818181818182
Learning rate:  0.5
Training Accuracy Score: 0.8581818181818182
Validation Accuracy Score: 0.6872727272727273
Learning rate:  0.75
Training Accuracy Score: 0.8860606060606061
Validation Accuracy Score: 0.6981818181818182
Learning rate:  1
Training Accuracy Score: 0.8860606060606061
Validation Accuracy Score: 0.7054545454545454


# Confusion matrix, Recall, and Precision

In [107]:
gb = GradientBoostingClassifier(n_estimators=20, learning_rate = 0.5, max_features=2, max_depth = 2, random_state = 0)
gb.fit(x_train_1, y_train_1)
predictions = gb.predict(x_validation_1)

print("Confusion Matrix:")
print(confusion_matrix(y_validation_1, predictions))

Confusion Matrix:
[[136  30]
 [ 57  52]]


In [108]:
print("Classification Report")
print(classification_report(y_validation_1, predictions))

Classification Report
              precision    recall  f1-score   support

           0       0.70      0.82      0.76       166
           1       0.63      0.48      0.54       109

    accuracy                           0.68       275
   macro avg       0.67      0.65      0.65       275
weighted avg       0.68      0.68      0.67       275



In [109]:
gb.score(x_validation_1, y_validation_1)

0.6836363636363636

# Receiver Operating Characteristic (ROC) and Area Under the Curve (AUC)

In [110]:
y_scores_gb = gb.decision_function(x_validation_1)
f_gb, t_gb, _ = roc_curve(y_validation_1, y_scores_gb)
roc_auc_gb = auc(f_gb, t_gb)

print("Area under ROC curve = {:}".format(roc_auc_gb))

Area under ROC curve = 0.733309384326296


In [111]:
roc_auc_score(y_test,y_pred_test)

0.49529707580555044